In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 546

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터
        ├─01.데이터
        │  ├─2_Validation
        │  │  ├─원천데이터
        │  │  │  └─VS1.zip | 8 MB | 62136
        │  │  └─라벨링데이터
        │  │      └─VL1.zip | 50 MB | 62135
        │  └─1_Training
        │      ├─라벨링데이터
        │      │  └─TL1.zip | 410 MB | 62133
        │      └─원천데이터
        │          └─TS1.zip | 67 MB | 62134
        └─02.저작도구
            ├─저작도구 소스코드.zip | 723 MB | 59162
            └─저작도구 설명서.zip | 4 MB | 59163


데이터 개요는 이 [페이지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=546)에서 찾아볼 수 있다. 라벨링데이터를 다운받아 분석을 진행한다.

In [3]:
!aihubshell -mode d -datasetkey 546 -filekey 62133

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     00
100  409M    0  409M    0     0  5321k      0 --:--:--  0:01:18 --:--:-- 7483k635k      0 --:--:--  0:00:26 --:--:-- 6907k0:01:08 --:--:-- 4773k
Request successful with HTTP status 200.
Download successful.
x 027.일상생활_및_구어체_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1_Training/라벨링데이터/TL1.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging TL1.zip in ./027.일상생활_및_구어체_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1_Training/라벨링데이터
병합이 완료 되었습니다. 


다운받은 데이터 디렉토리 및 결과 디렉토리 정의

In [4]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("027*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

데이터 압축파일 경로 정의

In [5]:
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("라벨링데이터"):
            archive_path = f"{current_path}/{file_name}"

압축 해제

In [6]:
data_dir = pathlib.Path(archive_path.strip(".zip"))
shutil.unpack_archive(
    filename=archive_path,
    extract_dir=data_dir,
    format="zip",
)

각 데이터는 json 형태로 저장되어 있다.

In [7]:
data_files = list(data_dir.glob("*/*/*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 12


데이터 속성값들. 카테고리별로 한국어-일본어 문장 페어가 다른 특징들과 함께 포함되어 있는 것을 확인할 수 있다. 이 데이터셋은 한-중 번역문도 포함하고 있기 때문에, 예를 들어 한-일 번역문만 사용하고자 한다면  (S_Code, T_Code)가 (ja-JP, ko-KR) 또는 (ko-KR, ja-JP)인 데이터만 사용해야 한다.

In [8]:
with open(data_files[0], "r") as file:
    documents = json.load(file)
pprint(documents[:3])

[{'1차수정': '이 과정을 탈아데닐화라고 합니다.',
  '2차수정': '',
  'ID': 1,
  'MT': '이 과정을 탈아데닐화라고 한다.',
  'Ratio': '94%',
  'S_Code': 'ja-JP',
  'S_Length': 17,
  'T_Code': 'ko-KR',
  'T_Length': 18,
  '대분류': '채팅',
  '소분류': '기타',
  '원문': 'この過程を脱アデニル化と呼びます。',
  '중분류': '답변',
  '최종번역문': '이 과정을 탈아데닐화라고 합니다.',
  '특수표현': 'X',
  '화자': 'X'},
 {'1차수정': '이 관계는 막다른 곳에 있어.',
  '2차수정': '',
  'ID': 2,
  'MT': '이 관계는 막혔다.',
  'Ratio': '81%',
  'S_Code': 'ja-JP',
  'S_Length': 13,
  'T_Code': 'ko-KR',
  'T_Length': 16,
  '대분류': '채팅',
  '소분류': '기타',
  '원문': 'この関係は行き詰まってる。',
  '중분류': '답변',
  '최종번역문': '이 관계는 막다른 곳에 있어.',
  '특수표현': 'X',
  '화자': 'X'},
 {'1차수정': '이 관계는, 아마 잘 안될 거야.',
  '2차수정': '',
  'ID': 3,
  'MT': '이 관계는 아마 잘 작동하지 않습니다.',
  'Ratio': '94%',
  'S_Code': 'ja-JP',
  'S_Length': 17,
  'T_Code': 'ko-KR',
  'T_Length': 18,
  '대분류': '채팅',
  '소분류': '기타',
  '원문': 'この関係は、たぶんうまくいかない。',
  '중분류': '답변',
  '최종번역문': '이 관계는, 아마 잘 안될 거야.',
  '특수표현': 'X',
  '화자': 'X'}]


전체 로드

In [9]:
corpus = []
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        documents = json.load(file)
    for document in documents:
        source = document.pop("S_Code")
        target = document.pop("T_Code")
        if source == "ko-KR" and target == "ja-JP":
            korean = document.pop("원문")
            japanese = document.pop("최종번역문")
            data = {
                "korean": korean,
                "japanese": japanese,
                "category_1st": document.get("대분류", None),
                "category_2nd": document.get("중분류", None),
                "category_3rd": document.get("소분류", None),
            }
            corpus.append(data)
        elif source == "ja-JP" and target == "ko-KR":
            korean = document.pop("최종번역문")
            japanese = document.pop("원문")
            data = {
                "korean": korean,
                "japanese": japanese,
                "category_1st": document.get("대분류", None),
                "category_2nd": document.get("중분류", None),
                "category_3rd": document.get("소분류", None),
            }
            corpus.append(data)

100%|███████████████████████████████████████████| 12/12 [00:08<00:00,  1.35it/s]


EDA를 위해 데이터프레임으로 변환

In [10]:
eda_dataset = pl.DataFrame(corpus).sample(fraction=1, shuffle=True)
print(f"Number of documents in the corpus : {eda_dataset.height:,}")

Number of documents in the corpus : 1,200,000


### 카테고리별 문서 수 분포

카테고리별 문서 수 분포는 아래와 같았다. 사용하려는 필요에 따라 이 중 특정 카테고리에서 발생한 문서들은 제외해도 무방할 듯 하다.

In [11]:
(
    eda_dataset
    .group_by(["category_1st", "category_2nd", "category_3rd"])
    .len("num_documents")
    .sort(by=["category_1st", "category_2nd", "category_3rd"])
)

category_1st,category_2nd,category_3rd,num_documents
str,str,str,u32
"""일상생활""","""스포츠""","""관람""","62,500"
"""일상생활""","""스포츠""","""운동""","37,500"
"""일상생활""","""여행""","""공항, 기내""","30,000"
"""일상생활""","""여행""","""관광""","30,000"
"""일상생활""","""여행""","""쇼핑""","30,000"
"""일상생활""","""여행""","""숙소""","30,000"
"""일상생활""","""여행""","""음식점""","30,000"
"""일상생활""","""일상""","""영화 및 여가""","100,000"
"""일상생활""","""일상""","""일반""","50,000"


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 한국어 토큰이 약 670만개 가량 포함된 corpus임을 확인했다.

In [12]:
eda_dataset = eda_dataset.with_columns(pl.col("korean").str.replace(r"\s+", " "))
num_tokens = (
    eda_dataset
    .with_columns((pl.col("korean").str.count_matches(" ") + 1).alias("num_tokens"))
    .select("num_tokens")
)
print(f"Number of tokens in the Korean corpus : {num_tokens['num_tokens'].sum():,}")

Number of tokens in the Korean corpus : 6,739,216


문서당 토큰 수 분포

In [13]:
num_tokens.select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
u32,i32,i32,i32,i32,i32,u32
1,3,4,5,7,10,55


토큰 한개짜리 문장들도 outlier라기보단 구어체에서 흔히 나타날 수 있는 문장임을 확인

In [14]:
(
    eda_dataset
    .with_columns((pl.col("korean").str.count_matches(" ") + 1).alias("num_tokens")).filter(pl.col("num_tokens") == 1)
    .sample(10)
    .select(["korean", "japanese"])
)

korean,japanese
str,str
"""도와주실래요?""","""助けてくれますか。"""
"""운동하자""","""運動しよう。"""
"""찾아볼게요.""","""探してみますね。"""
"""흠.""","""ふむ。"""
"""워크리스트모니터스레드가정지되었습니다.""","""ワークリスト モニタ スレッドが停止しました。"""
"""생일선물입니다.""","""お誕生日プレゼントになります。"""
"""ラーメンは日本で最もポピュラーな料理の一つです。""","""ラーメンは日本で最もポピュラーな料理の一つです。"""
"""짜증나요.""","""イライラしています。"""
"""왜지?""","""何でだろう。"""


### 마스킹 토큰 존재 여부 확인

다른 AI 허브 데이터같이 비식별화가 필요한 텍스트에 마스크가 적용되어 있는지 확인했다. 없다.

In [15]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([가-힣]+\))").alias("mask_pattern"))
    .filter(pl.col("mask_pattern").is_not_null())
    .group_by("mask_pattern")
    .len("pattern_count")
    .sort(by="pattern_count", descending=True)
    .head(10)
)

mask_pattern,pattern_count
str,u32
"""(주)""",20
"""(금)""",4
"""(월)""",4
"""(토)""",4
"""(화)""",2
"""(목)""",2
"""(웃음)""",2
"""(작성)""",2
"""(모니터링)""",2


### 한국어에 일본어 텍스트가 포함된 경우

하지만 한국어 안에 일본어가 포함된 데이터가 일부 있었다.

In [16]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([一-龯ぁ-んァ-ン]+\))").alias("jp_pattern"))
    .filter(pl.col("jp_pattern").is_not_null())
    .group_by("jp_pattern")
    .len("pattern_count")
    .sort(by="pattern_count", descending=True)
    .head(10)
)

jp_pattern,pattern_count
str,u32
"""(田中)""","2,747"
"""(太郎)""","1,861"
"""(良一)""","1,448"
"""(日本)""","1,213"
"""(東京)""",821
"""(鈴木)""",467
"""(隆志)""",373
"""(花子)""",371
"""(紀子)""",290


In [17]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([一-龯ぁ-んァ-ン]+\))").alias("jp_pattern"))
    .filter(pl.col("jp_pattern").is_not_null())
    .sample(10)
    .select(["korean", "japanese"])
)

korean,japanese
str,str
"""타로(太郎)는 아주 비싼 카메라를 샀다.""","""太郎はとても高価なカメラを買った。"""
"""기이치(喜一)와 아키라(彰)는 침대에서 이야기하고 있습니다.""","""喜一と彰はベッドで話しています。"""
"""런던(ロンドン)의 동물원은 영국(イギリス)에서 제일 오래됐어요.""","""ロンドンの動物園はイギリスで一番古いです。"""
"""카지노에서 빌(ビル)은 어떻게 하고 있어?""","""カジノでビルはどうしてる？"""
"""교토(京都) 역사 지구는 유네스코 세계유산에 등재되어 있어요.""","""京都歴史地区はユネスコの世界遺産に登録されています。"""
"""간신히 일어나 줘서 기뻐, 쿠리모토(栗本).""","""やっと立ち上がってくれてうれしいよ、栗本。"""
"""일본(日本)에서의 생활은 어떻습니까?""","""日本での生活はいかがですか。"""
"""저는 매주 3번 타로(太郎)와 함께 일본어를 공부합니다.""","""私は毎週 3 回、太郎と一緒に日本語を勉強します。"""
"""어느 날 유스케(祐介)는 매우 늦게까지 컴퓨터 게임을 하고 있었습니다.""","""ある日、祐介は非常に遅くまでコンピューター ゲームをしていました。"""


한국어에는 한국어만 포함시키기 위해 이런 패턴은 전부 제거하는 전처리를 적용했다.

In [18]:
eda_dataset = eda_dataset.with_columns(pl.col("korean").str.replace(r"(\([一-龯ぁ-んァ-ン]+\))", ""))

## 데이터 저장

EDA를 마친 corpus를 앞서 정의한 `result_dir`에 같은 다운받은 데이터셋과 동일한 이름으로 저장

In [19]:
result_path = result_dir.joinpath(f"{archive_dir.name}.parquet")
eda_dataset.write_parquet(result_path)

로컬 디렉토리 정리

In [20]:
os.system(f"rm -rf {archive_dir}")

0

저장된 데이터셋에는 한국어 문장(`korean`), 이에 대응하는 일본어 문장(`japanese`)과 함께 해당 문장의 카테고리가 저장되어 있다.

In [21]:
eda_dataset.head(10)

korean,japanese,category_1st,category_2nd,category_3rd
str,str,str,str,str
"""당신은 편의점에서 일한 적이 있나요?""","""あなたはコンビニで働いたことがありますか。""","""채팅""","""답변""","""음악"""
"""한 500개 정도?""","""およそ500個くらい？""","""해외영업""","""제조""","""계약"""
"""조회하고 말씀해 주세요?""","""照会してからお話しくださいか？""","""채팅""","""문의""","""항공편"""
"""혹시 이 메뉴를 먹어보려면 오래 걸릴까요?""","""すみません、このメニューを食べてみるには長くかかりますか？""","""일상생활""","""여행""","""음식점"""
"""요리책을 침대 속에서 읽는 게 은근한 기쁨이야.""","""料理の本をベッドの中で読むのが密かな喜びなんだよね。""","""채팅""","""답변""","""음식"""
"""네네 그럼 바로 입금해주시길 바랍니다.""","""はいはい。ではすぐに振り込みをお願いします。""","""해외영업""","""기타""","""사전영업"""
"""저희 아버지는 좋은 일을 하세요.""","""私の父は良い仕事をしています。""","""일상생활""","""일상""","""일반"""
"""백분율이 높을수록 압축률이 낮고 화질이 우수함을 나타냅니다.""","""パーセンテージが高いほど、圧縮率が低く、画質が優れていることを示します。""","""해외영업""","""금융""","""사전영업"""
"""우선 고객님 성함먼저 여쭤보겠습니다.""","""まず、お客様のお名前から伺います。""","""해외영업""","""금융""","""사전영업"""
